In [ ]:
import sys #this probably isn't necessary, but i was using it so i could exit code to test it before it errored
import pyodbc
import pandas as pd
import spacy

#load nlp library via path
nlp = spacy.load(r'en_core_web_sm') 

conn_string = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\Muhammad Yasir\Desktop\KINS_Text_Test_11_Python_v1.accdb;')

#connector to database
conn = pyodbc.connect(conn_string)

#pandas sql start
sql='select name, uuid, short_description from Kins_Target_List_1 order by name'

#create first datatable from Access DB
dt = pd.read_sql(sql, conn)

dt.head()

display (dt)

#dt['short_description']=dt['short_description'].apply(str)

dt['tokenized_descriptions'] = dt['short_description'].apply(lambda x: list(nlp(x)))

display (dt)


#filter tokens
def filter_stopwords(tokens):
    return [token.text for token in tokens if token.is_alpha and not token.is_stop]
dt['tokenized_descriptions_cleaned'] = dt['tokenized_descriptions'].apply(
    lambda x: 
        list(filter_stopwords(x))
            )

display (dt)

#convert to doc
def make_doc(tokens):
    return [nlp.make_doc(text) for text in tokens]
dt['tokenized_descriptions_cleaned_asDoc'] = dt['tokenized_descriptions_cleaned'].apply(
    lambda x:
        list(make_doc(x))
            )

display (dt)

#Starting with Matcher
from spacy.matcher import PhraseMatcher

#initialize the matcher with vocab
matcher = PhraseMatcher(nlp.vocab)

#define pattern for matching; THis should ultimately be pulled from the same database
#my_lexicon = [{"LOWER": {"IN":["Saas", "Cloud"]}}]
#my_lexicon_terms = ["saas", "cloud", "network"]
#my_lexicon_patterns = [nlp.make_doc(text) for text in my_lexicon_terms]

#Reading from the database "Lexicon".

query_terms = 'select Term from Lexicon'
dt_terms = pd.read_sql(query_terms, conn)

#Extracting terms and converting them to doc.
lexicon_terms = dt_terms.Term.to_list()
my_lexicon_patterns = [nlp.make_doc(text) for text in lexicon_terms]

#define the token matcher
matcher.add('Kins Lexicon Match', None, *my_lexicon_patterns)

#run the matcher
def piped_matcher(docs):
    fin_result = []
    
    for doc in docs:
        end_result = []
        character_matches = matcher(doc)
        for match_id, start, end in character_matches:
            span = doc[start:end]
            end_result.append(span)
        fin_result.append(end_result)
    return fin_result

dt['Matches'] = dt['tokenized_descriptions_cleaned_asDoc'].apply(
    lambda x: 
        piped_matcher(x)
)      
display(dt)